# Use taxonomic read classifications from MEGAN6 Phylum-specific extractions to convert from FastA to FastQs - Arthropoda and Unassigned Reads

## Input data are here:

FastAs: https://gannet.fish.washington.edu/Atumefaciens/20230726-mmag-read_extraction/

Trimmed-FastQs: https://gannet.fish.washington.edu/Atumefaciens/20230301-mmag-trimmed_rnaseq_from_noaa/

## List computer specs

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Sun Jul 30 16:32:07 PDT 2023
------------



No LSB modules are available.


Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.6 LTS
Release:	18.04
Codename:	bionic

------------
HOSTNAME: 
raven

------------
Computer Specs:

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              48
On-line CPU(s) list: 0-47
Thread(s) per core:  2
Core(s) per socket:  24
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Gold 5220R CPU @ 2.20GHz
Stepping:            7
CPU MHz:             1000.055
CPU max MHz:         4000.0000
CPU min MHz:         1000.0000
BogoMIPS:            4400.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            36608K
NUMA node0 CPU(s):   0-47
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdts

## Set variables

In [2]:
# Set data directories
%env RNAseq_dir=/home/shared/8TB_HDD_01/sam/data/M_magister/RNAseq
%env fasta_dir=/home/shared/8TB_HDD_01/sam/analyses/20230726-mmag-read_extraction
%env wd=/home/shared/8TB_HDD_01/sam/analyses

%env species=mmag
%env phyla=arthropoda-and-unassigned

# Programs
%env seqtk=/home/shared/seqtk-1.4/seqtk

env: RNAseq_dir=/home/shared/8TB_HDD_01/sam/data/M_magister/RNAseq
env: fasta_dir=/home/shared/8TB_HDD_01/sam/analyses/20230726-mmag-read_extraction
env: wd=/home/shared/8TB_HDD_01/sam/analyses
env: species=mmag
env: phyla=arthropoda-and-unassigned
env: seqtk=/home/shared/seqtk-1.4/seqtk


## Extract phylum-specific reads from trimmed FastQ files


Use FastA IDs from MEGAN6 taxonomic read extraction FastAs to pull out appropriate reads from each phylum (Arthropoda and Alveolata). This is performed because MEGAN6 strips paired read ID after the first space. As such, the resulting read extractions using MEGAN end up with a FastA file containing two reads with identicial headers. Not sure if this will cause any downstream issues (i.e. with Trinity) where paired end data is used, so playing it safe and using the truncated IDs to pull FastQs with complete sequence headers for use in subsequent data wrangling.

### Create list of FastA IDs

In [3]:
%%bash

timestamp=$(date +%Y%m%d)

    
# Make new directory and change to that directory ("$_" means use previous command's argument)
mkdir --parents "${wd}"/"${timestamp}"."${species}"-megan_reads \
&& cd "$_" || exit

# Set seqtk list filename
seqtk_list=${timestamp}.${species}.seqtk.read_id.list

######################################################
# Create FastA IDs list to use for sequence extraction
######################################################
for fasta in ${fasta_dir}/*.fasta
do
    echo "Pulling FastA IDs from ${fasta}"
    echo ""
    grep ">" "${fasta}" | awk 'sub(/^>/, "")'
done | sort -u >> "${seqtk_list}"
    
    
echo ""
echo "Finished with FastA ID extraction."



Finished with FastA ID extraction.


### Insepct FastQ IDs file

In [4]:
%%bash
timestamp=$(date +%Y%m%d)
cd "${wd}"/"${timestamp}"."${species}"-megan_reads

head "${timestamp}.${species}.seqtk.read_id.list"


GWNJ-1012:589:GW2108041346th:3:1101:10004:10676
GWNJ-1012:589:GW2108041346th:3:1101:10004:13683
GWNJ-1012:589:GW2108041346th:3:1101:10004:14215
GWNJ-1012:589:GW2108041346th:3:1101:10004:1532
GWNJ-1012:589:GW2108041346th:3:1101:10004:17597
GWNJ-1012:589:GW2108041346th:3:1101:10004:18380
GWNJ-1012:589:GW2108041346th:3:1101:10004:18850
GWNJ-1012:589:GW2108041346th:3:1101:10004:2002
GWNJ-1012:589:GW2108041346th:3:1101:10004:2096


### Generate MD5 checksums for input FastAs

In [5]:
%%bash
timestamp=$(date +%Y%m%d)
cd "${wd}"/"${timestamp}"."${species}"-megan_reads

# Get checksums for FastAs
for fasta in ${RNAseq_dir}/*.fasta
do
    echo ""
    echo "Recording MD5 checksum for ${fasta}."
    md5sum ${fasta_dir}/*.fasta | tee --append input-fasta-checksums.md5
    echo ""
done

echo ""
echo "Moving on to read extractions..." 
echo ""


Recording MD5 checksum for /home/shared/8TB_HDD_01/sam/data/M_magister/RNAseq/*.fasta.
cf5c9f9f4ca98e49ad92eb57f4a0de0b  /home/shared/8TB_HDD_01/sam/analyses/20230726-mmag-read_extraction/CH01-06.trimmed.R1-MEGAN_summarized_reads-arthropoda_NA.fasta
66df16af1a44a7491d053cb1995c5880  /home/shared/8TB_HDD_01/sam/analyses/20230726-mmag-read_extraction/CH01-06.trimmed.R2-MEGAN_summarized_reads-arthropoda_NA.fasta
fd81ac48f2dfbb1e9baee926265660e6  /home/shared/8TB_HDD_01/sam/analyses/20230726-mmag-read_extraction/CH01-14.trimmed.R1-MEGAN_summarized_reads-arthropoda_NA.fasta
12facf9917b418952bc732e5772d7f3a  /home/shared/8TB_HDD_01/sam/analyses/20230726-mmag-read_extraction/CH01-14.trimmed.R2-MEGAN_summarized_reads-arthropoda_NA.fasta
124e669eae1bbc652134ce50a2e4d2f7  /home/shared/8TB_HDD_01/sam/analyses/20230726-mmag-read_extraction/CH01-22.trimmed.R1-MEGAN_summarized_reads-arthropoda_NA.fasta
87aeab58b98dc88143ca3fbfc9dd063a  /home/shared/8TB_HDD_01/sam/analyses/20230726-mmag-read_extract

## Extract FastQs Using `seqtk`

In [6]:
%%bash

timestamp=$(date +%Y%m%d)

# Set seqtk list filename
seqtk_list=${timestamp}.${species}.seqtk.read_id.list

# Set output FastQ filenames
prefix=${timestamp}.${species}
R1_suffix=megan_R1.fq
R2_suffix=megan_R2.fq

cd "${wd}"/"${timestamp}"."${species}"-megan_reads

######################################################
# Extract corresponding R1 and R2 reads using seqtk FastA ID list
######################################################
for fastq in "${RNAseq_dir}"/*R1*.gz
do
    # Strip path from filename
    fastq_nopath=${fastq##*/}
                     
    # Get sample ID from FastQ filename
    sample=$(echo "${fastq_nopath}" | awk -F "." '{print $1"."$2}')
 
    R1_out="${prefix}.${sample}.${R1_suffix}"
        
    echo "Extracting R1 reads from ${fastq}."
    echo ""
    echo "Writing R1 reads to ${R1_out}"
    echo ""
      
    # Use seqtk to pull out desired FastQ reads
  	${seqtk} subseq "${fastq}" "${seqtk_list}" >> "${R1_out}"
    
    # Gzip output file
    echo ""
    echo "Compressing ${R1_out}."
    gzip "${R1_out}"
    echo ""
done
    
echo ""
echo "Done with R1 read extractions"
echo "-------------------------------------"
echo ""

for fastq in "${RNAseq_dir}"/*R2*.gz
do
    # Strip path from filename
    fastq_nopath=${fastq##*/}
                     
    # Get sample ID from FastQ filename
    sample=$(echo "${fastq_nopath}" | awk -F "." '{print $1"."$2}')
          
    # Set output filename    
    R2_out="${prefix}.${sample}.${R2_suffix}"
        
   
    echo "Extracting R2 reads from ${fastq}."
    echo ""
    echo "Writing R2 reads to ${R2_out}"
    echo ""
      
    # Use seqtk to pull out desired FastQ reads
  	${seqtk} subseq "${fastq}" "${seqtk_list}" >> "${R2_out}"
    
    # Gzip output file
    echo ""
    echo "Compressing ${R2_out}."
    gzip "${R2_out}"
    echo ""

done
    
echo "-------------------------------------"
echo ""
                   
# Print working directory and list files
pwd
ls -ltrh

Extracting R1 reads from /home/shared/8TB_HDD_01/sam/data/M_magister/RNAseq/CH01-06.trimmed.R1.fastq.gz.

Writing R1 reads to 20230730.mmag.CH01-06.trimmed.megan_R1.fq


Compressing 20230730.mmag.CH01-06.trimmed.megan_R1.fq.

Extracting R1 reads from /home/shared/8TB_HDD_01/sam/data/M_magister/RNAseq/CH01-14.trimmed.R1.fastq.gz.

Writing R1 reads to 20230730.mmag.CH01-14.trimmed.megan_R1.fq


Compressing 20230730.mmag.CH01-14.trimmed.megan_R1.fq.

Extracting R1 reads from /home/shared/8TB_HDD_01/sam/data/M_magister/RNAseq/CH01-22.trimmed.R1.fastq.gz.

Writing R1 reads to 20230730.mmag.CH01-22.trimmed.megan_R1.fq


Compressing 20230730.mmag.CH01-22.trimmed.megan_R1.fq.

Extracting R1 reads from /home/shared/8TB_HDD_01/sam/data/M_magister/RNAseq/CH01-38.trimmed.R1.fastq.gz.

Writing R1 reads to 20230730.mmag.CH01-38.trimmed.megan_R1.fq


Compressing 20230730.mmag.CH01-38.trimmed.megan_R1.fq.

Extracting R1 reads from /home/shared/8TB_HDD_01/sam/data/M_magister/RNAseq/CH03-04.trimmed.R1.f

## Generate checksums

In [7]:
%%bash
timestamp=$(date +%Y%m%d)

cd "${wd}"/"${timestamp}"."${species}"-megan_reads

for file in *
do
  md5sum ${file} | tee --append checksums.md5
done

e6047623dc885392b3ae816f094552b4  20230730.mmag.CH01-06.trimmed.megan_R1.fq.gz
40b096dc26066a0d715d05e4bde36806  20230730.mmag.CH01-06.trimmed.megan_R2.fq.gz
401d6206dcd62c8b682536eef19ee4cc  20230730.mmag.CH01-14.trimmed.megan_R1.fq.gz
dcfe7a42861fa7406e1cc16522584045  20230730.mmag.CH01-14.trimmed.megan_R2.fq.gz
e9c1b9a7d3afb9bdd4beaff2c6ddb0fa  20230730.mmag.CH01-22.trimmed.megan_R1.fq.gz
7491813d90b50e08e3cefca8e1a1d7fc  20230730.mmag.CH01-22.trimmed.megan_R2.fq.gz
99bf9577b46bf2dfca8b8ac042e179ed  20230730.mmag.CH01-38.trimmed.megan_R1.fq.gz
4be920ddd78862fe25a46e66235faad4  20230730.mmag.CH01-38.trimmed.megan_R2.fq.gz
2f85cd2b10061b89011123d3bfa58d4f  20230730.mmag.CH03-04.trimmed.megan_R1.fq.gz
d220a8d18d8a67bfba19410879464f43  20230730.mmag.CH03-04.trimmed.megan_R2.fq.gz
a20b5ac49bdf3e17f18d0dbc1b60d6d2  20230730.mmag.CH03-15.trimmed.megan_R1.fq.gz
7cf21ab8f759a50183d31584aac1d1e3  20230730.mmag.CH03-15.trimmed.megan_R2.fq.gz
be8e50bf2c6747671ab4c2f2c3cbc5fd  20230730.mmag.CH03